In [1]:
import sdv
import pandas as pd

print(sdv.version.public)

1.11.0


In [2]:
# import zipfile
# import os
# zip_path = 'drive-download-20240323T140328Z-001.zip'
# extract_path = os.getcwd()

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)


### Loading the data

In [3]:
#viewing the data we have
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
validation = pd.read_csv('validation.csv')
hmof = pd.read_csv('hmof.csv')

# if  a single df
# data = pd.read_csv('data.csv')

<li> I think this notebook will help in synthesizing our data
<li> Our data should not be divided (i mean we should have a single df that contains (train,test, validation) after we do the split)
<li>Problem : we need to try and find a way of making our hmof ids unique and same (cos this ids .cifs are the ones available , different onece will cause a problem)

In [4]:
train.head()

,hMOF-21761,1.4469189829363254,1.0116009116784799,-1.2196517457139742,5.5258512661635395,-1.8613410854197348
0,hMOF-6005116,1.321756,0.810930,-2.301835,4.730921,-1.781057
1,hMOF-5015132,1.749200,1.178655,-0.678929,6.767228,0.474263
2,hMOF-27705,1.749200,1.178655,-0.754667,7.120283,-2.842239
3,hMOF-5072656,1.446919,1.178655,-1.289789,5.132263,-1.197971
4,hMOF-5076184,1.446919,1.011601,-1.284149,6.075576,-2.642411


column_names = ['id','lcd', 'pld', 'void_fraction', 'surface_area_m2g','surface_area_m2cm3', 'pressure_0.01(bar)']


### Meta data

In [ ]:
from sdv import metadata


metadata.detect_from_dataframes(data)

In [ ]:
print('Auto detected data:\n')
metadata.visualize()

   set  'pii=True' if you want to anonymize that column in your corrections
   


In [ ]:
#in a case where there is mistake in the auto detection of metadata
metadata.update_columns(
    column_names=['age', 'transactions', 'session_length'],
    sdtype='numerical',
    computer_representation='Float'
)

In [ ]:
#setting primary key
metadata.set_primary_key(column_name='guest_email')

In [ ]:
#validating metadata
metadata.validate()

In [ ]:
metadata.validate_data(data=data)

In [ ]:
metadata.visualize()

In [ ]:
from sdv.metadata import SingleTableMetadata

metadata.save_to_json(filepath='my_metadata_v1.json')




In [ ]:
metadata = SingleTableMetadata.load_from_json(filepath='my_metadata_v1.json')

### Creating a synthesizer
An SDV synthesizer is an object that we  use to create synthetic data. It learns patterns from the real data and replicates them to generate synthetic data.



In [ ]:
#fast method
from sdv.lite import SingleTablePreset

# synthesizer = SingleTablePreset(metadata, name='FAST_ML')
# synthesizer.fit(data)

# synthetic_data = synthesizer.sample(num_rows=10)


from sdv.single_table import GaussianCopulaSynthesizer

# Step 1: Create the synthesizer
synthesizer = GaussianCopulaSynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(data)

# Step 3: Generate synthetic data
synthetic_data = synthesizer.sample(num_rows=100)

In [ ]:
#evaluating our the data with the real ones
from sdv.evaluation.single_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=data,
    synthetic_data=synthetic_data,
    metadata=metadata
)

In [ ]:
#measure the data quality or the statistical similarity between the real and synthetic data.
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data,
    synthetic_data,
    metadata
)

In [ ]:
quality_report.get_details('Column Shapes')

In [ ]:
#checking the correlation of columns in the synthesizin

from sdv.evaluation.single_table import get_column_pair_plot

fig = get_column_pair_plot(
    real_data=real_data,
    synthetic_data=synthetic_data,
    column_names=['checkin_date', 'checkout_date'],
    metadata=metadata
)

fig.show()

In [ ]:
#saving the synthesizing model for future use
from sdv.single_table import SingleTablePreset

synthesizer.save('my_synthesizer.pkl')

synthesizer = SingleTablePreset.load('my_synthesizer.pkl')